In [1]:
include("decision_trees.jl")
using .DecisionTreeAlgorithm
using Combinatorics
using DataStructures
using PrettyTables

In [2]:
n = 3

set_partitions = collect(partitions(1:n))
partition2id = Dict(partition => id for (id, partition) in enumerate(set_partitions))
decision_trees = [
    DecisionTree{Int}([DecisionTreeStep([1, 2, 3, 4], 1)]),
    DecisionTree{Int}([DecisionTreeStep([1, 2, 3, 4], 2)]),
    DecisionTree{Int}([
        DecisionTreeStep([2], 1),
        DecisionTreeStep([1], 2),
        DecisionTreeStep([3], 1), 
        DecisionTreeStep([4], 2)
    ]),
    
    DecisionTree{Int}([
        DecisionTreeStep([2], 2),
        DecisionTreeStep([1], 1),
        DecisionTreeStep([3], 2), 
        DecisionTreeStep([4], 1)
    ]),
    DecisionTree{Int}([
        DecisionTreeStep([3], 1),
        DecisionTreeStep([1], 2),
        DecisionTreeStep([2], 1), 
        DecisionTreeStep([4], 2)
    ]),
    DecisionTree{Int}([
        DecisionTreeStep([3], 2),
        DecisionTreeStep([1], 1),
        DecisionTreeStep([2], 2), 
        DecisionTreeStep([4], 1)
    ]),
    DecisionTree{Int}([
        DecisionTreeStep([1], 1),
        DecisionTreeStep([2, 3], 2),
        DecisionTreeStep([4], 1), 
    ]),
    DecisionTree{Int}([
        DecisionTreeStep([1], 2),
        DecisionTreeStep([2, 3], 1),
        DecisionTreeStep([4], 2), 
    ]),
];

In [3]:
using ProgressMeter


valid_partition_tuples = Vector{Int}[]
for g1_partition in set_partitions
    g1_condensation = zeros(Bool, n+1, n+1)
    for block in g1_partition, (u, v) in combinations(block, 2)
        g1_condensation[u, v] = true
        g1_condensation[v, u] = true
    end
    colorings = apply_decision_tree.(Ref(g1_condensation), decision_trees[2:end])
    @showprogress for partition_tuple in Iterators.product(ntuple(_ -> set_partitions, length(decision_trees)-1)...)
        partition_tuple = collect(partition_tuple)
        if check_partition_compatibility(g1_condensation, colorings, partition_tuple)
            push!(valid_partition_tuples, map(p->partition2id[p], vcat([g1_partition], partition_tuple)))
        end
    end
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01K
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:01
Progress: 100%|█████████████████████████████████████████| Time: 0:00:16


# Recover inequality (5) for 3-hyperedges

In [6]:
using Gurobi
using JuMP

opt_gurobi = optimizer_with_attributes(
    Gurobi.Optimizer,
    "OutputFlag" => 0,
    "LogFile" => "gurobi.log",
    "FeasibilityTol" => 1e-9
)

N = length(set_partitions)
D = length(decision_trees)
model = Model(opt_gurobi)
@variable(model, -1 ≤ phi[1:D÷2, 1:N, 1:N] ≤ 1, Int)
for partition_ids in valid_partition_tuples
    constr = AffExpr(0.0)
    for d in 1:D÷2
        add_to_expression!(constr, phi[d, partition_ids[2*d-1], partition_ids[2*d]])
    end
    @constraint(model, constr ≥ 0)
end

abc = 0.369
a_b_c = 0.369
ab_c = (1 - abc - a_b_c) / 3
ac_b = (1 - abc - a_b_c) / 3
bc_a = (1 - abc - a_b_c) / 3

mu = [abc, bc_a, ab_c, ac_b, a_b_c]
@constraint(model, phi[1, :, :] .== 0)
@objective(model, Min, sum(phi[d, i, j] * mu[i] * mu[j] for d in 1:D÷2, i in 1:N, j in 1:N))
optimize!(model)
@show objective_value(model)
Phi = round.(Int, value.(phi));

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-16
Set parameter FeasibilityTol to value 1e-09
objective_value(model) = -0.0013474444444444705


In [7]:
inequality = sum(Phi, dims=1)[1, :, :]
headers = vcat([""], [partition_repr(set_partitions[i]) for i in 1:N])
println("Inequality: ")
pretty_table(
    hcat(headers[2:end], inequality + inequality'),
    header=headers,
    alignment=[:l, :r, :r, :r, :r, :r],
    backend = Val(:html)
)


Inequality: 


,123,12|3,13|2,1|23,1|2|3
123,0,1,1,0,-1
12|3,1,0,2,1,0
13|2,1,2,0,1,0
1|23,0,1,1,2,1
1|2|3,-1,0,0,1,0


In [8]:
println("Potentials: ")
for d = 1:D÷2
    headers[1] = "S$(2d-1) / S$(2d)"
    pretty_table(
        hcat(headers[2:end], Phi[d, :, :]),
        header=headers,
        alignment=[:l, :r, :r, :r, :r, :r],
        backend = Val(:html)
    )
end

Potentials: 


S1 / S2,123,12|3,13|2,1|23,1|2|3
123,0,0,0,0,0
12|3,0,0,0,0,0
13|2,0,0,0,0,0
1|23,0,0,0,0,0
1|2|3,0,0,0,0,0


S3 / S4,123,12|3,13|2,1|23,1|2|3
123,-1,0,-1,0,0
12|3,0,0,0,0,0
13|2,1,1,1,1,1
1|23,1,0,1,0,0
1|2|3,0,0,0,0,0


S5 / S6,123,12|3,13|2,1|23,1|2|3
123,0,-1,0,-1,-1
12|3,1,1,1,1,1
13|2,-1,-1,-1,-1,-1
1|23,0,1,0,1,1
1|2|3,0,0,0,0,0


S7 / S8,123,12|3,13|2,1|23,1|2|3
123,1,1,1,1,1
12|3,0,-1,0,0,0
13|2,1,1,0,1,1
1|23,-1,-1,-1,0,0
1|2|3,-1,-1,-1,0,0


# Recover $e_2$ inequality (6)

In [10]:
using Gurobi
using JuMP

opt_gurobi = optimizer_with_attributes(
    Gurobi.Optimizer,
    "OutputFlag" => 0,
    "LogFile" => "gurobi.log",
    "FeasibilityTol" => 1e-9
)

N = length(set_partitions)
D = length(decision_trees)
model = Model(opt_gurobi)
@variable(model, -1 ≤ phi[1:D÷2, 1:N, 1:N] ≤ 1, Int)
for partition_ids in valid_partition_tuples
    constr = AffExpr(0.0)
    for d in 1:D÷2
        add_to_expression!(constr, phi[d, partition_ids[2*d-1], partition_ids[2*d]])
    end
    @constraint(model, constr ≥ 0)
end

@constraint(model, phi[1, :, :] .== 0)
ineq = sum(phi, dims=1)[1, :, :]
E = [
    0 0 0 0 1
    0 0 -1 -1 0
    0 -1 0 -1 0
    0 -1 -1 0 0
    1 0 0 0 0
]
@constraint(model, ineq + ineq' .== E)
optimize!(model)
@show primal_status(model)
Phi = round.(Int, value.(phi));

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-16
Set parameter FeasibilityTol to value 1e-09
primal_status(model) = MathOptInterface.FEASIBLE_POINT


In [11]:
println("Potentials: ")
for d = 1:D÷2
    headers[1] = "S$(2d-1) / S$(2d)"
    pretty_table(
        hcat(headers[2:end], Phi[d, :, :]),
        header=headers,
        alignment=[:l, :r, :r, :r, :r, :r],
        backend = Val(:html)
    )
end

Potentials: 


S1 / S2,123,12|3,13|2,1|23,1|2|3
123,0,0,0,0,0
12|3,0,0,0,0,0
13|2,0,0,0,0,0
1|23,0,0,0,0,0
1|2|3,0,0,0,0,0


S3 / S4,123,12|3,13|2,1|23,1|2|3
123,1,0,1,1,0
12|3,1,1,1,1,1
13|2,1,1,1,1,1
1|23,1,-1,-1,0,1
1|2|3,1,1,1,1,1


S5 / S6,123,12|3,13|2,1|23,1|2|3
123,0,1,0,0,1
12|3,0,0,-1,0,-1
13|2,0,0,0,0,0
1|23,0,1,1,1,-1
1|2|3,0,0,0,0,0


S7 / S8,123,12|3,13|2,1|23,1|2|3
123,-1,-1,-1,-1,-1
12|3,-1,-1,-1,-1,-1
13|2,-1,-1,-1,-1,-1
1|23,-1,-1,-1,-1,-1
1|2|3,0,0,-1,0,-1
